## Import libraries

In [1]:
import gc
import pickle
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_log_error

## Prepare data for model training

In [2]:
with open("../input/workation-price-prediction-preprocess-data/Workation_Dataset.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

train_df = train_df[~((train_df['Per Person Price'] < 7.3) | (np.exp(train_df['Per Person Price']) > 150000))].copy()

del processed_data
gc.collect()

31

In [3]:
Xtrain = train_df.loc[:, train_df.columns != 'Per Person Price'].copy()
Ytrain = train_df['Per Person Price'].copy()
Ytrain_strat = pd.qcut(train_df['Per Person Price'].values, q=10, labels=range(0,10))
Xtest = test_df.copy()

print("Xtrain: {} \nYtrain: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

Xtrain: (20989, 4728) 
Ytrain: (20989,) 
Xtest: (9000, 4728)


0

## Build and validate the model

In [4]:
FOLD = 10
NUM_SEED = 3

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

oof_score = 0
y_pred_meta_cb = np.zeros((Ytrain.shape[0], NUM_SEED))
y_pred_final_cb = np.zeros((Xtest.shape[0], NUM_SEED))
counter = 0


for sidx, seed in enumerate(seeds):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain.values, Ytrain_strat)):
        counter += 1

        train_x, train_y = Xtrain.iloc[train], Ytrain.iloc[train]
        val_x, val_y = Xtrain.iloc[val], Ytrain.iloc[val]

        model = CatBoostRegressor(
            objective='RMSE',
            eval_metric='RMSE',
            num_boost_round=8000,
            learning_rate=0.014,
            reg_lambda=0.0487,
            bootstrap_type='Poisson',
            subsample=0.922,
            max_depth=7, 
            grow_policy='Lossguide',
            min_data_in_leaf=6, 
            max_leaves=957,
            task_type='GPU',
            verbose=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  early_stopping_rounds=200, verbose=100)

        y_pred = model.predict(val_x)
        y_pred_meta_cb[val, sidx] += y_pred
        y_pred_final_cb[:, sidx] += model.predict(Xtest)
        
        score = np.sqrt(mean_squared_log_error(val_y, y_pred))
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_final_cb = y_pred_final_cb / float(FOLD)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

0:	learn: 0.5346268	test: 0.5299756	best: 0.5299756 (0)	total: 125ms	remaining: 16m 41s
100:	learn: 0.2295885	test: 0.2404821	best: 0.2404821 (100)	total: 10s	remaining: 13m 4s
200:	learn: 0.1618547	test: 0.1884406	best: 0.1884406 (200)	total: 19.5s	remaining: 12m 38s
300:	learn: 0.1405449	test: 0.1767420	best: 0.1767420 (300)	total: 28.8s	remaining: 12m 16s
400:	learn: 0.1309610	test: 0.1725924	best: 0.1725924 (400)	total: 35.2s	remaining: 11m 6s
500:	learn: 0.1257080	test: 0.1705228	best: 0.1705228 (500)	total: 40.9s	remaining: 10m 12s
600:	learn: 0.1214249	test: 0.1691175	best: 0.1691175 (600)	total: 46.1s	remaining: 9m 28s
700:	learn: 0.1181344	test: 0.1681014	best: 0.1681014 (700)	total: 51.2s	remaining: 8m 53s
800:	learn: 0.1150739	test: 0.1673418	best: 0.1673418 (800)	total: 57.1s	remaining: 8m 33s
900:	learn: 0.1123625	test: 0.1667856	best: 0.1667752 (897)	total: 1m 2s	remaining: 8m 11s
1000:	learn: 0.1097808	test: 0.1663425	best: 0.1663338 (996)	total: 1m 7s	remaining: 7m 49s


In [5]:
np.savez_compressed('./CB_Meta_Features.npz',
                    y_pred_meta_cb=y_pred_meta_cb, 
                    oof_score=oof_score,
                    y_pred_final_cb=y_pred_final_cb)

In [6]:
df = pd.DataFrame(y_pred_final_cb)
df.head()

,0,1,2
0,9.847647,9.852360,9.851337
1,9.331621,9.321807,9.327517
2,8.628285,8.626055,8.623000
3,8.864918,8.864986,8.858597
4,9.964543,9.965843,9.962442
